### Imports

In [77]:
from scipy import ndimage
import cv2 as cv
import numpy as np
import os
import time

In [78]:
def show_image(title, image):
    image = cv.resize(image, (0, 0), fx=0.2, fy=0.2)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Functia de detectie a colturilor patratului mare din tabla

In [79]:
def detectare_colturi(contours):
    top_left = None
    top_right = None
    bottom_right = None
    bottom_left = None

    gasit = False
        # print(s_min)
    for cont in contours:
        area = cv.contourArea(cont)

        # print(area)
        if 160000 < area < 170000:

            top_left = None
            bottom_right = None
            for point in cont.squeeze():
                if top_left is None or point[0] + point[1] < top_left[0] + top_left[1]:
                    top_left = point

                if bottom_right is None or point[0] + point[1] > bottom_right[0] + bottom_right[1] :
                    bottom_right = point

            diff = np.diff(cont.squeeze(), axis = 1)
            top_right = cont.squeeze()[np.argmin(diff)]
            bottom_left = cont.squeeze()[np.argmax(diff)]

            gasit = True
            # peri = cv.arcLength(cont, True)
            # cv.drawContours(img_final, cont, -1, (0, 100, 0), 3)
            # aprox = cv.approxPolyDP(cont, 0.01 * peri, True)
            # x, y, w, h = cv.boundingRect(aprox)
    # cv.imshow("colturi", img_final)
    if top_left is None or top_right is None or bottom_left is None or bottom_right is None:
        return False, 0, 0, 0, 0

    return gasit, top_left, top_right, bottom_left, bottom_right

### Functia de cropare a patratului

In [80]:
def crop_img(img_final, top_left, top_right, bottom_left, bottom_right):

    # 15 * w, h imagini piese cropate
    width = 1800
    height = 1950

    pts1 = np.float32([top_left, top_right, bottom_left, bottom_right])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)

    img_crop = cv.warpPerspective(img_final, matrix, (width, height))
    img_result = img_crop.copy()

    linii_verticale=[]
    for i in range(0,height + 1,120):
        l= [(i, 0), (i, height - 1)]
        linii_verticale.append(l)

    linii_orizontale=[]
    for i in range(0,height + 1,130):
        l= [(0, i), (width - 1, i)]
        linii_orizontale.append(l)

    ###Vizionarea patratului cropat, si impartit pe linii/col
    # for line in  linii_verticale :
    #     cv.line(img_crop, line[0], line[1], (0, 255, 0), 5)
    # for line in  linii_orizontale :
    #     cv.line(img_crop, line[0], line[1], (0, 0, 255), 5)
    # img_crop = cv.resize(img_crop, (0, 0), fx=0.4, fy=0.4)
    # cv.imshow("img cropata si impartita in linii", img_crop)

    return img_result

### Functii de modificare a imaginii initiale cu operatii morfologice

In [81]:
### fereastra de proprietati custom
def empty(a):
    pass


def createGUI():
    cv.namedWindow("trackBars", cv.WINDOW_NORMAL)
    # cv.resizeWindow("trackBarks", 640, 240)
    cv.createTrackbar("Hue Min", "trackBars", 0, 179, empty)
    cv.createTrackbar("Hue Max", "trackBars", 179, 179, empty)
    cv.createTrackbar("Sat Min", "trackBars", 0, 255, empty)
    # intre 110 si 140
    cv.createTrackbar("Sat Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("Val Min", "trackBars", 0, 255, empty)
    cv.createTrackbar("Val Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("MedBlur", "trackBars", 5, 21, empty)
    cv.createTrackbar("GausBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("Sharp alp", "trackBars", 12, 20, empty)
    cv.createTrackbar("Sharp bet", "trackBars", 5, 20, empty)
    cv.createTrackbar("Sharp gam", "trackBars", 0, 20, empty)
    cv.createTrackbar("tresmin", "trackBars", 67, 255, empty)
    cv.createTrackbar("tresmax", "trackBars", 255, 255, empty)
    cv.createTrackbar("cannymin", "trackBars", 0, 500, empty)
    cv.createTrackbar("cannymax", "trackBars", 0, 500, empty)
    cv.createTrackbar("erode", "trackBars", 3, 5, empty)


def track_bar_values():
    h_min = cv.getTrackbarPos("Hue Min", "trackBars")
    h_max = cv.getTrackbarPos("Hue Max", "trackBars")
    s_min = cv.getTrackbarPos("Sat Min", "trackBars")
    s_max = cv.getTrackbarPos("Sat Max", "trackBars")
    v_min = cv.getTrackbarPos("Val Min", "trackBars")
    v_max = cv.getTrackbarPos("Val Max", "trackBars")
    median = cv.getTrackbarPos("MedBlur", "trackBars")
    gaussian = cv.getTrackbarPos("GausBlur", "trackBars")
    alpha = cv.getTrackbarPos("Sharp alp", "trackBars")
    beta = cv.getTrackbarPos("Sharp bet", "trackBars")
    gama = cv.getTrackbarPos("Sharp gam", "trackBars")
    canny_min = cv.getTrackbarPos("cannymin", "trackBars")
    canny_max = cv.getTrackbarPos("cannymax", "trackBars")
    treshold_min = cv.getTrackbarPos("tresmin", "trackBars")
    treshold_max = cv.getTrackbarPos("tresmax", "trackBars")
    erode = cv.getTrackbarPos("erode", "trackBars")

    return h_min, h_max, s_min, s_max, v_min, v_max, median, gaussian, alpha, beta, gama, canny_min, canny_max, treshold_min, treshold_max, erode

In [82]:
def extrage_tabla(img_originala):

    img = cv.resize(img_originala, (0, 0), fx=0.2, fy=0.2)
    # img = img_originala

    # img = img_originala
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    # createGUI()
    # h_min, h_max, s_min, s_max, v_min, v_max, median, gaussian, alpha, beta, gama, canny_min, canny_max, treshold_min, treshold_max, erode = track_bar_values()

    h_min = 0
    h_max = 179
    s_min = 110
    s_max = 255
    v_min = 0
    v_max = 255
    median = 1
    gaussian = 1
    alpha = 12
    beta = 8
    gama = 0
    treshold_min = 5
    treshold_max = 255
    canny_min = 0
    canny_max = 0
    erode = 3

    while True:
        # filtrare HSV
        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        mask = cv.inRange(img_hsv, lower, upper)
        img_result = cv.bitwise_and(img, img, mask=mask)

        # HSV to GRAY
        h, s, v = cv.split(img_result)
        img_result = v

        # filtre morfologice
        img_m_blur = cv.medianBlur(img_result, median)
        img_g_blur = cv.GaussianBlur(img_result, (7, 7), gaussian)
        img_sharpened = cv.addWeighted(img_m_blur, alpha / 10, img_g_blur, (-1) * beta / 10, gama)
        _, thresh = cv.threshold(img_sharpened, treshold_min, treshold_max, cv.THRESH_BINARY)
        kernel = np.ones((erode, erode), np.uint8)
        thresh = cv.erode(thresh, kernel)
        edges = cv.Canny(thresh, canny_min, canny_max)

        # functia de detectare a contururilor
        contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # print(len(contours))

        gasit, top_left, top_right, bottom_left, bottom_right = detectare_colturi(contours)

        # Daca gasim aria unui contur care se potriveste cu tabla noastra, se opreste din cautat, daca nu, marim saturatia pentru a rezolva problemele cu lumina
        if gasit:
            # cv.rectangle(img_final, tuple(top_left), tuple(bottom_right), (255, 255, 0), 2)
            pass
        else:
            s_min += 2
            if s_min > 150:
                break
            continue

        ### imagini cu rezultatul fiecarei operatii
        # cv.imshow("Thresh", thresh)
        # cv.imshow("Edges", edges)
        # cv.imshow("Original", img)
        # cv.imshow("BlurG", img_g_blur)
        # cv.imshow("BlurM", img_m_blur)
        # cv.imshow("sharpened", img_sharpened)
        # cv.imshow("Result", img_result)
        ### rezultatul cu patratul final
        # cv.circle(img_final,tuple(top_left),10,(0,0,255),-1)
        # cv.circle(img_final,tuple(top_right),10,(0,0,255),-1)
        # cv.circle(img_final,tuple(bottom_left),10,(0,0,255),-1)
        # cv.circle(img_final,tuple(bottom_right),10,(0,0,255),-1)

        img_cropata = crop_img(img, top_left, top_right, bottom_left, bottom_right)
        # cv.imshow("AFISARE TABLA", img_cropata)
        return img_cropata

### Functia de detectie a pieselor

In [83]:
def pozitii_piese(img_cropata):
    matrice_piese = np.full(shape=(15, 15), fill_value=False)

    img = cv.resize(img_cropata, (0, 0), fx=0.4, fy=0.4)
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    h_min = 68
    h_max = 123
    s_min = 0
    s_max = 161
    v_min = 165
    v_max = 255
    treshold_min = 5
    treshold_max = 255


    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv.inRange(img_hsv, lower, upper)
    img_result = cv.bitwise_and(img, img, mask=mask)

    # HSV to GRAY
    h, s, v = cv.split(img_result)
    img_result = v

    # filtre morfologice
    _, thresh = cv.threshold(img_result, treshold_min, treshold_max, cv.THRESH_BINARY)

    ### imagini cu rezultatul fiecarei operatii
    # cv.imshow("Thresh", thresh)
    # cv.imshow("Original", img)
    # cv.imshow("Result", img_result)


    ### vrem sa trecem prin fiecare patratel din imaginea binara si sa numaram pixelii albi, daca patratul contine mai mult de 30% pixeli albi, atunci contine o piesa
    w, h = thresh.shape
    w_patrat = w // 15
    h_patrat = h // 15
    border=5
    nr_pixeli_patrat = (w_patrat - border) * (h_patrat - border)
    for i in range(0, 15):
        for j in range(0, 15):
            patrat = thresh[i*w_patrat+border:(i+1)*w_patrat-border,
                            j*h_patrat+border:(j+1)*h_patrat-border]
            pixeli_pozitivi = np.sum(patrat)
            if pixeli_pozitivi/255 >= nr_pixeli_patrat * 30 / 100:
                matrice_piese[i][j] = True

    return matrice_piese

In [84]:
def pozitii_mutari(matrice_pozitii_piese, matrice_pozitii_piese_mutare):
    mutari = []
    for i in range(15):
        for j in range(15):
            if matrice_pozitii_piese[i][j] != matrice_pozitii_piese_mutare[i][j]:
                mutare = (i + 1, letters[j + 1])
                mutari.append(mutare)
    return mutari

### Functia de recunoastere a pieselor

In [85]:
def litere_mutari(mutari_func, img_crop):
    litere = []
    for mutare in mutari_func:
        i, j = mutare
        j = ord(j) - ord('A')
        i -= 1

        patch = img_crop[i*130:(i+1)*130, j*120:(j+1)*120]
        # cv.imshow("patch", patch)
        img = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)
        _, patchthresh = cv.threshold(img, 160, 255, cv.THRESH_BINARY)
        # cv.imshow("patchthresh", patchthresh)
        # cv.waitKey(0)

        piesa_string = ""
        max = -np.inf

        path_piese = "imagini_piese/"
        img_piese = os.listdir(path_piese)
        for piese in img_piese:
            i = 0
            swap = False
            piesa=cv.imread(path_piese + piese)

            piesa = cv.cvtColor(piesa, cv.COLOR_BGR2GRAY)
            _, piesa = cv.threshold(piesa, 111, 255, cv.THRESH_BINARY)
            # cv.imshow("templatethresh", piesa)
            # cv.waitKey(0)
            while i < 360:
                piesa_rotita = ndimage.rotate(piesa, i)

                h_piesa_rotita, w_piesa_rotita = piesa_rotita.shape[:2]
                y_centru_piesa_rotita, x_centru_piesa_rotita = h_piesa_rotita // 2, w_piesa_rotita // 2

                raza = 40
                templatethresh = piesa_rotita[y_centru_piesa_rotita-raza:y_centru_piesa_rotita+raza, x_centru_piesa_rotita-raza:x_centru_piesa_rotita+raza]

                res = cv.matchTemplate(patchthresh, templatethresh,  cv.TM_CCOEFF_NORMED)
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
                if(max_val > max):
                    max = max_val
                    piesa_string = piese

                # print(max)
                if swap == False and i > 4:
                    i += 360 - (2 * i)
                    swap = True
                else:
                    i += 2
                # if(max > 0.8):
                #     break
        # cv.imshow("patch", patch)
        # asd = piesa_string.split(".")[0]
        # print(f"{max} - > {asd}")
        litere.append(piesa_string.split(".")[0])
        # cv.waitKey(1)
        # time.sleep(0.2)
    return litere

### Functia de scriere in fisier a pozitiilor ocupate pentru fiecare mutare

In [86]:
letters = {i:chr(i+64) for i in range(1,16)}
print(letters)

{1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M', 14: 'N', 15: 'O'}


In [87]:
def scriere_fisier_mutari(file, mutari, litere):

    nume_fisier = file.split(".")[0] + ".txt"
    path = "txtbuilder/"
    nume_fisier = path + nume_fisier
    f = open(nume_fisier, "w")
    string = ""
    for mutare, litera in zip(mutari, litere):
        string += str(mutare[0])
        string += mutare[1]
        string += " "
        if(litera != "JOKER"):
            string += litera
        else:
            string += "?"
        string += "\n"
    string += "0"
    f.write(string)

In [88]:
time_start = time.time()
path = 'antrenare/'

files = os.listdir(path)
# files = ["3_09.jpg", "3_10.jpg"]

matrice_pozitii_piese = np.full(shape=(15, 15), fill_value=False)
counter = 0

for file in files:

    # cream matrice de pozitii noua pentru fiecare joc
    if "_01.jpg" in file:
        matrice_pozitii_piese = np.full(shape=(15, 15), fill_value=False)

    # verificam ca fisierul sa fie de tip jpg
    if file[len(file) - 3:len(file)] == "jpg":
        print(f"{file} -> {counter} din {len(files)//2}")
        counter += 1
        img = cv.imread(path + file)

        img_cropata = extrage_tabla(img)

        time_end = time.time()
        print(f"Timpul de rulare al programului dupa extragere al tabelei:{time_end - time_start} s")

        matrice_pozitii_piese_mutare = pozitii_piese(img_cropata)

        time_end = time.time()
        print(f"Timpul de rulare al programului dupa pozitii piese:{time_end - time_start} s")

        mutari = pozitii_mutari(matrice_pozitii_piese, matrice_pozitii_piese_mutare)

        time_end = time.time()
        print(f"Timpul de rulare al programului dupa pozitii mutari:{time_end - time_start} s")

        litere_mutare = litere_mutari(mutari, img_cropata)
        # cv.imshow("imagine cropata", img_cropata)

        time_end = time.time()
        print(f"Timpul de rulare al programului dupa litere mutari({len(mutari)} litere):{time_end - time_start} s")

        scriere_fisier_mutari(file, mutari, litere_mutare)

        time_end = time.time()
        print(f"Timpul de rulare al programului dupa scrierea fisierului:{time_end - time_start} s")

        matrice_pozitii_piese = matrice_pozitii_piese_mutare

        # img_show = cv.resize(img_cropata, (0, 0), fx=0.3, fy=0.3)
        # cv.imshow("img", img_show)
        # cv.waitKey(0)

        # apasa p pentru a trece la urmatoarea poza
        # if cv.waitKey(0) & 0xFF == ord('p'):
        #     break

        # trece automat la urmatoarea poza dupa 0.3s
        # cv.waitKey(1)
        # time.sleep(0.3)
        # break

    cv.destroyAllWindows()
time_end = time.time()
print(f"Timpul de rulare al programului este:{time_end - time_start} s")


1_01.jpg -> 0 din 100
Timpul de rulare al programului dupa extragere al tabelei:0.1829972267150879 s
Timpul de rulare al programului dupa pozitii piese:0.1889958381652832 s
Timpul de rulare al programului dupa pozitii mutari:0.1889958381652832 s
Timpul de rulare al programului dupa litere mutari(5 litere):1.9229974746704102 s
Timpul de rulare al programului dupa scrierea fisierului:1.9229974746704102 s
1_02.jpg -> 1 din 100
Timpul de rulare al programului dupa extragere al tabelei:2.0859971046447754 s
Timpul de rulare al programului dupa pozitii piese:2.090996742248535 s
Timpul de rulare al programului dupa pozitii mutari:2.09199857711792 s
Timpul de rulare al programului dupa litere mutari(3 litere):3.035996198654175 s
Timpul de rulare al programului dupa scrierea fisierului:3.0369975566864014 s
1_03.jpg -> 2 din 100
Timpul de rulare al programului dupa extragere al tabelei:3.2069973945617676 s
Timpul de rulare al programului dupa pozitii piese:3.213027000427246 s
Timpul de rulare al 

### Test

In [89]:
163.401

163.401